In [11]:
# %cd 'drive/My Drive/VGGish'
# %ls
# !pip install librosa==0.7.1
# import librosa 
# librosa.__version__

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 3.6MB 38.7MB/s 
  Created wheel for librosa: filename=librosa-0.7.1-cp36-none-any.whl size=1610157 sha256=c7d2e68cf991ec26d8eff4fb730bd25d8ee793d84b9b837c3d4e6e3163d77819
  Stored in directory: /root/.cache/pip/wheels/07/36/47/a9a4d151332cbdaec564500af9704a0ad862cf554dcf4bfda0
Successfully built librosa
  Found existing installation: numba 0.40.1
    Uninstalling numba-0.40.1:
      Successfully uninstalled numba-0.40.1
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3


In [9]:
import tensorflow as tf
import sys
import librosa
import os
import copy
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt
from perceptual_loss import perceptual_loss_fn
%matplotlib inline


Looks Good To Me!





Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.


INFO:tensorflow:Restoring parameters from vggish_model.ckpt



In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Load style and content

In [0]:
CONTENT_FILENAME = "inputs/content/bhulaDena.mp3"
STYLE_FILENAME = "inputs/style/spongebob.mp3"
method = "stft"

In [0]:
display(Audio(CONTENT_FILENAME))
display(Audio(STYLE_FILENAME))

In [0]:
N_FFT = 2048
def read_audio_spectum(file_name, method="stft"):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X, N_FFT))
    if method == "mfcc":
        S = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
    elif method == "mel":
        S = librosa.feature.melspectrogram(X, sr=sample_rate)
    else:
        S = stft
    S = np.log1p(np.abs(S))  
    return S, sample_rate


In [0]:
a_content, fs = read_audio_spectum(CONTENT_FILENAME, method)
a_style, fs = read_audio_spectum(STYLE_FILENAME, method)
N_SAMPLES = min(a_style.shape[1], a_content.shape[1])
N_CHANNELS = min(a_style.shape[0], a_content.shape[0])

a_style = a_style[:N_CHANNELS,:N_SAMPLES]
a_content = a_content[:N_CHANNELS,:N_SAMPLES]

In [0]:
a_content.shape

### Visualize spectrograms for content and style tracks

In [0]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Content')
plt.imshow(a_content)
plt.subplot(1, 2, 2)
plt.title('Style')
plt.imshow(a_style)
plt.show()

### Compute content and style feats

In [0]:
N_FILTERS = 4096
pool_width = 2

a_content_tf = np.ascontiguousarray(a_content.T[None,None,:,:])
a_style_tf = np.ascontiguousarray(a_style.T[None,None,:,:])

std = np.sqrt(2) * np.sqrt(2.0 / ((N_CHANNELS + N_FILTERS) * 11))
kernel = np.random.randn(1, 11, N_CHANNELS, N_FILTERS)*std
    
g = tf.Graph()
with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:

    x = tf.placeholder('float32', [1,1,N_SAMPLES,N_CHANNELS], name="x")

    kernel_tf = tf.constant(kernel, name="kernel", dtype='float32')
    conv = tf.nn.conv2d(
        x,
        kernel_tf,
        strides=[1, 1, 1, 1],
        padding="VALID",
        name="conv")
    
#     max_pool = tf.nn.max_pool(
#         conv,
#         ksize=[1,1,1,pool_width],
#         strides = [1,1,1,pool_width],
#         padding="VALID",
#         name="max_pool")
    # max_pool = tf.reduce_max(conv, reduction_indices=[pool_width], keep_dims=True)

    
    net = tf.nn.relu(conv)

    content_features = net.eval(feed_dict={x: a_content_tf})
    style_features = net.eval(feed_dict={x: a_style_tf})
    
    features = np.reshape(style_features, (-1, N_FILTERS))
    style_gram = np.matmul(features.T, features) / N_SAMPLES

### Optimize

In [0]:
from sys import stderr

ALPHA= 1e-2
iterations = 300
result = None
with tf.Graph().as_default() as graph, tf.Session() as sess:

    x = tf.Variable(np.random.randn(1,1,N_SAMPLES,N_CHANNELS).astype(np.float32)*1e-3, name="x")

    kernel_tf = tf.constant(kernel, name="kernel", dtype='float32')
    conv = tf.nn.conv2d(
        x,
        kernel_tf,
        strides=[1, 1, 1, 1],
        padding="VALID",
        name="conv")
    
#     max_pool = tf.nn.max_pool(
#         conv,
#         ksize=[1,1,1,pool_width],
#         strides = [1,1,1,pool_width],
#         padding="VALID",
#         name="max_pool")
    
    # max_pool = tf.reduce_max(conv, reduction_indices=[pool_width], keep_dims=True)
    sess.run(tf.initialize_all_variables())
    
    net = tf.nn.relu(conv)

    content_features = net.eval(feed_dict={x: a_content_tf})
    style_features = net.eval(feed_dict={x: a_style_tf})
    
    features = np.reshape(style_features, (-1, N_FILTERS))
    style_gram = np.matmul(features.T, features) / N_SAMPLES

    result_temp = (graph.get_tensor_by_name("x:0")).eval()
    a = np.zeros_like(a_content)
    a[:N_CHANNELS,:] = np.exp(result_temp[0,0].T) - 1
    inp1 = librosa.istft(a)

    result_temp = a_content
    a = np.zeros_like(a_content)
    a[:N_CHANNELS,:] = np.exp(result_temp[0,0].T) - 1
    inp2 = librosa.istft(a)

    cnt_loss = perceptual_loss_fn(inp1, inp2 ,fs)

    content_loss = ALPHA * 2 * (cnt_loss)

    style_loss = 0

    _, height, width, number = map(lambda i: i.value, net.get_shape())

    size = height * width * number
    feats = tf.reshape(net, (-1, number))
    gram = tf.matmul(tf.transpose(feats), feats)  / N_SAMPLES
    style_loss = 2 * tf.nn.l2_loss(gram - style_gram)

     # Overall loss
    loss = content_loss + style_loss

    opt = tf.contrib.opt.ScipyOptimizerInterface(
          loss, method='L-BFGS-B', 
          options={'maxiter': iterations})
        
    
    print('Started optimization.')
    opt.minimize(sess)

    print('Final loss:', loss.eval())
    result = x.eval()
print(result)

In [0]:
result

### Invert spectrogram and save the result

In [0]:
a = np.zeros_like(a_content)
a[:N_CHANNELS,:] = np.exp(result[0,0].T) - 1

if method == "mfcc":
    x = librosa.feature.inverse.mfcc_to_audio(a)
elif method == "mel":
    x = librosa.feature.inverse.mel_to_stft(a)
    x = librosa.istft(x)
else:    
    x = librosa.istft(a)

OUTPUT_FILENAME = 'outputs/out.wav'
librosa.output.write_wav(OUTPUT_FILENAME, x, fs)

In [0]:
print(OUTPUT_FILENAME)
display(Audio(OUTPUT_FILENAME))

### Visualize spectrograms

In [0]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.title('Content')
plt.imshow(a_content)
plt.subplot(1,3,2)
plt.title('Style')
plt.imshow(a_style)
plt.subplot(1,3,3)
plt.title('Result')
plt.imshow(a)
plt.show()